In [2]:
import pandas as pd

chd1 = pd.read_csv("~/HIDDEN-GEM/CHD1/CHD1_gem.csv", nrows=10000)

In [3]:
chd1

,smiles,score
0,[O-]S(=O)(=O)c(c1N)cc(S([O-])(=O)=O)c(c12)ccc(...,-79.548950
1,[O-]S(=O)(=O)c(c1N)cc(S([O-])(=O)=O)c(c12)ccc(...,-76.969788
2,c1cnccc1Cn2nc(c(c23)cnc(n3)NCCC)-c4cnc(cc4)N5C...,-49.904549
3,CN1CCC2(CC1)CC(=NOC1CCCCC1)C1=CC=CC=C1O2,-49.661621
4,CCN1C=C(N=C2SC=C(C)N2CC2=CC(S(C)(=O)=O)=CC=C2)...,-48.614803
...,...,...
9995,CCN1N=CC(C2=CSC(=NCCC(N)=O)N2CCC2=NC(C)=CC=C2)=N1,-33.629707
9996,CC(C)C1=CN=C(CN2C=NC3=C(C4=C(CCCCC4)S3)C2=O)O1,-33.629402
9997,CCc1ccccc1C(=O)Nc2cc(CN3CCCC3)cc(c2)N4CCC(CC4)...,-33.629051
9998,COC(=O)C1=CC(N=C2SC=C(C3=C(C)C=CS3)N2C[C@@H]2C...,-33.628925


In [4]:
from rdkit import Chem
mols = [Chem.MolFromSmiles(x) for x in chd1['smiles']]

In [5]:
from argo.gen_models.f_rag.fusion.slicer import MolSlicerForSAFEEncoder
import safe as sf

slicer = MolSlicerForSAFEEncoder(shortest_linker=True)


/nas/longleaf/home/shuhang/miniconda3/envs/f-rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
sf.encode(mols[0], slicer=slicer)

'Cc1cc(-c2ccc5c(C)c2)ccc1/N=N/c1ccc2c(S(=O)(=O)[O-])cc(S(=O)(=O)[O-])c(N)c2c1O.c16ccc2c(S(=O)(=O)[O-])cc(S(=O)(=O)[O-])c(N)c2c1O.N\\5=N/6'

In [9]:
# Encode all molecules and collect fragments
all_fragments = []
for mol in mols:
    try:
        safe_str = sf.encode(mol, slicer=slicer)
        fragments = safe_str.split('.')
        all_fragments.extend([sf.decode(frag, remove_dummies=False) for frag in fragments])
    except:
        pass

# Count frequency of each fragment
from collections import Counter
fragment_counts = Counter(all_fragments)

In [10]:
fragment_counts

Counter({'C#CCCN=c1scc([*:4])n1[*:3]': 560,
         'Cc1csc(=N[*:3])n1[*:4]': 287,
         'S(C[*:4])[*:3]': 270,
         'S(C[*:5])[*:4]': 224,
         'c1([*:4])ccccc1': 214,
         'N(=c1scc(C)n1[*:4])[*:3]': 185,
         'C(C[*:4])[*:3]': 169,
         'C(S[*:5])[*:4]': 146,
         'C(=CC(=O)[*:4])[*:3]': 145,
         'c1([*:3])ccccc1': 133,
         'n1([*:3])c(C)csc1=N[*:4]': 132,
         'c1([*:4])cnn(C)c1': 100,
         'C(=C[*:5])[*:4]': 98,
         'C(=C[*:4])[*:3]': 95,
         'C(=O)(C=C[*:4])[*:3]': 94,
         'C(S[*:4])[*:3]': 92,
         'C(=CC(=O)[*:5])[*:4]': 84,
         'C1([*:4])CC1': 81,
         'N(=c1scc(C)n1[*:5])[*:4]': 79,
         'N(C(=O)[*:5])[*:4]': 77,
         'C(C[*:5])[*:4]': 68,
         'c1([*:5])ccccc1': 67,
         'N(C(=O)[*:4])[*:3]': 67,
         'c1([*:4])csc(=NCCC(N)=O)n1[*:3]': 66,
         'C(=O)(C=C[*:5])[*:4]': 64,
         'c1([*:4])ccnn1C': 64,
         'COc1ccccc1-c1nnc([*:3])n1N': 60,
         'n1([*:4])nnnc1[*:3]': 5

: 